Use of Native Function for LLM output validation

In [1]:
!python -m pip install beautifulsoup4 requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 2.1 MB/s eta 0:00:00a 0:00:01


In [2]:
import requests
from bs4 import BeautifulSoup

def scrape_google_maps_list(url):
    # Make a request to the URL
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print("Failed to retrieve the webpage.")
        return

    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find the elements you're interested in. This step may require inspecting the webpage to determine the correct selectors.
    places = soup.find_all("div", class_="place-name-selector")  # This is just a hypothetical class. You'd have to inspect the page to get the correct one.

    for place in places:
        print(place.text)

In [3]:
# Call the function with the public link to your list
scrape_google_maps_list("https://maps.app.goo.gl/Mg5u7qChmWNgmoLr9")

Initialize kernel

In [3]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

kernel = sk.Kernel()
api_key, org_id = sk.openai_settings_from_dot_env()
kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

Create inline Semantic Function to generate random JSON about a topic via LLM
The TOPIC will be provided as first argument to the function.

In [14]:
sk_prompt ="generate a random json array with at most 10 elements and 5 properties each, each payload about {{$INPUT}}, the output must be complete a parsable json, max lenght 512, only print the output"

random_json_function = kernel.create_semantic_function(sk_prompt, 'json', 'myrandom', 'generate random json about a topic',512)
res = await random_json_function.invoke_async("restaurant")
print(res.result)

[{"name": "Restaurant A", "address": "123 Main St", "city": "New York", "rating": 4.5, "cuisine": "Italian"},
{"name": "Restaurant B", "address": "456 Elm St", "city": "Los Angeles", "rating": 4.2, "cuisine": "Mexican"},
{"name": "Restaurant C", "address": "789 Oak St", "city": "Chicago", "rating": 4.7, "cuisine": "American"},
{"name": "Restaurant D", "address": "321 Pine St", "city": "San Francisco", "rating": 4.9, "cuisine": "Japanese"},
{"name": "Restaurant E", "address": "654 Maple St", "city": "Miami", "rating": 4.3, "cuisine": "Seafood"},
{"name": "Restaurant F", "address": "987 Cedar St", "city": "Seattle", "rating": 4.6, "cuisine": "Indian"},
{"name": "Restaurant G", "address": "135 Walnut St", "city": "Austin", "rating": 4.4, "cuisine": "Thai"},
{"name": "Restaurant H", "address": "246 Birch St", "city": "Denver", "rating": 4.8, "cuisine": "Steakhouse"},
{"name": "Restaurant I", "address": "579 Ash St", "city": "Boston", "rating": 4.1, "cuisine": "French"},
{"name": "Restauran

Now we will register the ValidatorSkill into the kernel
Then we will extract the ValidateJson function from the skill and use it to validate the LLM output

In [6]:
validator_skill = kernel.import_skill(ValidatorSkill(), 'ValidatorSkill')
json_validator_function = validator_skill["ValidateJson"]

validation_result = json_validator_function(res.result)
print(validation_result.result)

# Add something to break the parses
validation_result = json_validator_function(res.result + "asd")
print(validation_result.result)

[ERROR]Invalid JSON generated.[/ERROR]
